In [14]:
#Import library
from numpy import array
from numpy import hstack
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D 

In [2]:
#Define split function to convert sequence into input and output
def split_sequence(sequence, n_steps):
  X, y = list (), list ()
  for i in range(len(sequence)):
    end_ix = i + n_steps
    if end_ix > len(sequence)-1:
      break
    seq_x, seq_y = sequence[i: end_ix, :], sequence[end_ix, :]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

In [3]:
#Define sequence
#Define sequence dataset
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
in_seq2 = array([in_seq1[i] ** 2 for i in range(len(in_seq1))])
in_seq3 = array([in_seq1[i] * 2 for i in range(len(in_seq1))])
in_seq4 = array([in_seq1[i] + 5 for i in range(len(in_seq1))])
out_seq5 = array([in_seq1[i]+in_seq2[i]+in_seq3[i]+in_seq4[i] for i in range(len(in_seq1))])

In [4]:
#Convert into rows and columns structure / 2D structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
in_seq3 = in_seq3.reshape((len(in_seq3), 1))
in_seq4 = in_seq4.reshape((len(in_seq4), 1))
out_seq5 = out_seq5.reshape((len(out_seq5), 1))

In [5]:
#Horizontal stack
dataset = hstack((in_seq1, in_seq2, in_seq3, in_seq4, out_seq5))

In [6]:
#define n_steps
n_steps = 3

In [9]:
#Split into input (X) and output parallel (Y)
X, y = split_sequence(dataset, n_steps)

In [10]:
#Explore data X and y
print(X.shape, y.shape)
for i in range(len(X)):
  print(X[i], y[i])

(7, 3, 5) (7, 5)
[[  10  100   20   15  145]
 [  20  400   40   25  485]
 [  30  900   60   35 1025]] [  40 1600   80   45 1765]
[[  20  400   40   25  485]
 [  30  900   60   35 1025]
 [  40 1600   80   45 1765]] [  50 2500  100   55 2705]
[[  30  900   60   35 1025]
 [  40 1600   80   45 1765]
 [  50 2500  100   55 2705]] [  60 3600  120   65 3845]
[[  40 1600   80   45 1765]
 [  50 2500  100   55 2705]
 [  60 3600  120   65 3845]] [  70 4900  140   75 5185]
[[  50 2500  100   55 2705]
 [  60 3600  120   65 3845]
 [  70 4900  140   75 5185]] [  80 6400  160   85 6725]
[[  60 3600  120   65 3845]
 [  70 4900  140   75 5185]
 [  80 6400  160   85 6725]] [  90 8100  180   95 8465]
[[  70 4900  140   75 5185]
 [  80 6400  160   85 6725]
 [  90 8100  180   95 8465]] [  100 10000   200   105 10405]


In [11]:
#define n_feature
n_feature = X.shape[2]

In [24]:
#Separate output
y1 = y[:, 0].reshape((y.shape[0], 1))
y2 = y[:, 1].reshape((y.shape[0], 1))
y3 = y[:, 2].reshape((y.shape[0], 1))

In [18]:
visible = Input(shape = (n_steps, n_feature))
cnn = Conv1D(filters= 64, kernel_size=2, activation= 'relu')(visible)
cnn = MaxPooling1D(pool_size= 2)(cnn)
cnn = Flatten()(cnn)
cnn = Dense(50, activation= 'relu')(cnn)

In [19]:
#define output 1
output_1 = Dense(1)(cnn)

In [20]:
#define oupput 2
output_2 = Dense(1)(cnn)

In [21]:
#define output 3
output_3 = Dense(1)(cnn)

In [22]:
#Tile together 
model = Model(inputs = visible, outputs = [output_1, output_2, output_3])
model.compile(optimizer = 'adam', loss = 'mse')

In [25]:
#fit model
model.fit(X,[y1, y2, y3], epochs= 1000, verbose= 0)

In [27]:
#Explore the model
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 3, 5)]       0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 2, 64)        704         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 1, 64)        0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 64)           0           max_pooling1d_1[0][0]            
______________________________________________________________________________________________

In [32]:
#Demonstrate prediction
x_input = dataset[7:10, :]
x_input = x_input.reshape(1, n_steps, n_feature)
y_hat = model.predict(x_input, verbose = 0)
print(y_hat)

[array([[145.0355]], dtype=float32), array([[12137.401]], dtype=float32), array([[200.19827]], dtype=float32)]


In [35]:
predict = list(y_hat)
print(predict[1])

[[12137.401]]
